In [ ]:
General Statistics from The World Bank DataBank

In [ ]:
#import packages
import pandas as pd
import numpy as np

In [ ]:
#load data
factors = pd.read_csv("../climate_final/factors.csv")    #gdp, population, imports, exports
countries = pd.read_csv("../climate_final/countries.csv")    #country info

In [ ]:
#remove aggregates from countries df
countries = countries[countries.Region != 'Aggregates']
#select columns of interest in countries df
countries_reduced = countries[["Country code", "Country name", "Region"]]
#rename country name in countries df to accomplish merge
countries_renamed = countries_reduced.rename(columns={"Country name":"Country Name"})
#delete country code and time code column froms factors df
factors_reduced = factors.drop(columns=["Country Code", "Time Code"])
#merge datasets on country name
data = pd.merge(countries_renamed, factors_reduced, on="Country Name")

In [ ]:
#rename columns
data_renamed = data.rename(columns={'Country code': 'iso', 
                                    'Country Name': 'country', 
                                    'Region': 'region', 
                                    'Time': 'year', 
                                    'Time Code': 'year_code',
       'Population, total [SP.POP.TOTL]': 'pop_total',
       'Population growth (annual %) [SP.POP.GROW]': 'pop_growth',
       'GDP (constant 2010 US$) [NY.GDP.MKTP.KD]': 'gdp',
       'Imports of goods and services (constant 2010 US$) [NE.IMP.GNFS.KD]':'imports',
       'Exports of goods and services (current US$) [NE.EXP.GNFS.CD]':'exports',
       'Average precipitation in depth (mm per year) [AG.LND.PRCP.MM]':'avg_precip'})

In [ ]:
#use numpy na to remove empty rows
data_renamed["iso"].replace('nan', np.nan, inplace=True)
data_renamed["iso"].replace('NaN', np.nan, inplace=True)
data_renamed.dropna(axis=0, subset=['iso'], inplace=True)
#replace '..' with numpy na
data_renamed.replace('..', np.nan, inplace=True)
#make copy with clean name
factors = data_renamed.copy()
#make sure year column is numeric
factors['year'] = factors['year'].astype(int)

In [ ]:
#load data
emissions_csv = pd.read_csv("../climate_final/emissions.csv")    #emissions of GHGs by country
#reduce emissions dataset to data of interest
##columns
emissions_subset = emissions_csv[['iso_code', 'country', 'year', 'co2', 'methane', 
                                  'nitrous_oxide', 'total_ghg']]
##years
emissions_reduced = emissions_subset[emissions_subset['year']>=2000]
##renamed iso_code to iso
emissions_reduced = emissions_reduced.rename(columns={'iso_code':'iso'})
##make copy with clean name
emissions = emissions_reduced.copy()
emissions

In [ ]:
Emissions Data - Our World in Data

In [ ]:
#load data
emissions_csv = pd.read_csv("../climate_final/emissions.csv")    #emissions of GHGs by country
#reduce emissions dataset to data of interest
##columns
emissions_subset = emissions_csv[['iso_code', 'country', 'year', 'co2', 'methane', 
                                  'nitrous_oxide', 'total_ghg']]
##years
emissions_reduced = emissions_subset[emissions_subset['year']>=2000]
##renamed iso_code to iso
emissions_reduced = emissions_reduced.rename(columns={'iso_code':'iso'})
##make copy with clean name
emissions = emissions_reduced.copy()
emissions

In [ ]:
Extreme Weather Information - EM-DAT International Disaster Database

In [ ]:
#load data
disasters_csv = pd.read_csv("../climate_final/disasters.csv")    #database of occurences of natural disasters
#reduce disasters dataset to columns of interest
##columns
disasters_subset = disasters_csv[['Year','Country', 'ISO', 'Region', 'Continent', 
                                  'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 
                                  'Disaster Subtype', 'Disaster Subsubtype','Total Deaths', 
                                  'Total Affected', 'Latitude', 'Longitude']]
##years of interest
disasters_reduced = disasters_subset[disasters_subset['Year']>=2000]
##make column names lowercase
disasters_reduced = disasters_reduced.rename(columns=str.lower)
##make copy with clean name
disasters = disasters_reduced.copy()

In [ ]:
Temperature Data - Kaggle

In [ ]:
#load data
temps_bycity_csv = pd.read_csv("../climate_final/temps_bycity.csv")    #land temperatures by city
#remove average temperature uncertainty column
temps_bycity_reduced = temps_bycity_csv[['dt','AverageTemperature','City', 'Country', 
                                         'Latitude', 'Longitude']]
#rename columns
temps_bycity_renamed = temps_bycity_reduced.rename(columns={'dt':'date',
                                                       'AverageTemperature': 'avg_temp_c',
                                                       'City': 'city',
                                                       'Country': 'country',
                                                       'Latitude': 'latitude',
                                                       'Longitude':'longitude'})
#create a temperature column in farenheit
temps_bycity_renamed['avg_temp_f'] = [((x*9/5) + 32) for x in temps_bycity_renamed['avg_temp_c']]

In [ ]:
#create year and month columns
year_list = []
month_list = []
#for loop over date column
for date in temps_bycity_renamed['date']:
    split = date.split("-")
    y = pd.to_numeric(split[0])    #get year from string split
    m = pd.to_numeric(split[2])    #get month from string split
    year_list.append(y)    #append result to year list
    month_list.append(m)    #append result to month list
#turn lists in columns
temps_bycity_renamed['year'] = year_list
temps_bycity_renamed['month'] = month_list
#reorder columns
temps_bycity = temps_bycity_renamed[['year', 'month', 'date', 'city', 'country', 
                                     'avg_temp_c', 'avg_temp_f', 'latitude', 'longitude']]
#select years
temps_bycity = temps_bycity[temps_bycity['year']>=2000]

In [ ]:
##create a dataframe of averaged temps by country
country_list = temps_bycity['country'].unique()
yr_list = temps_bycity['year'].unique()
temps_f_list = []
temps_c_list = []
country_col = []
year_col = []
#loop through countries
for i in range(len(country_list)):
    #loop through years
    for j in range(len(yr_list)):
        #subset to particular country and year
        c = country_list[i]
        y = yr_list[j]
        subset = temps_bycity[(temps_bycity['country']==c) & (temps_bycity['year']==y)]
        #average all temperature readings from that year
        avg_f = subset['avg_temp_f'].mean()
        avg_c = subset['avg_temp_c'].mean()
        #add averages to lists
        temps_f_list.append(avg_f)
        temps_c_list.append(avg_c)
        #add country names and years to lists
        country_col.append(c)
        year_col.append(y)
    
#create dataframe
temps_bycountry = pd.DataFrame({"country": country_col,
                                "year": year_col,
                               "avg_temp_c": temps_c_list,
                               "avg_temp_f": temps_f_list})

In [ ]:
The last step to merge datasets together

In [ ]:
#create dataframes that match each research questions
##Question 1
q1 = pd.merge(factors, emissions, on=['country','year', 'iso'])
q1_df = pd.merge(q1, temps_bycountry, on=['country', 'year'])
##Question 2
q2_df = pd.merge(emissions, temps_bycountry, on=['country','year'])
##Question 3
factors_subset = factors[['country', 'region', 'year', 'pop_total', 'pop_growth']]
q3_df = pd.merge(factors_subset, temps_bycountry, on=['country','year'])
##Question 4
factors_subset2 = factors[['country', 'region', 'year', 'avg_precip']]
q4 = pd.merge(factors_subset2, temps_bycountry, on=['country', 'year'])
q4_df = pd.merge(q4, emissions, on=['country', 'year'])

In [ ]:
emissions.to_csv('../climate_final/emissions_clean.csv', index=False)
factors.to_csv('../climate_final/factors_clean.csv', index=False)
disasters.to_csv('../climate_final/disasters_clean.csv', index=False)
temps_bycity.to_csv('../climate_final/temps_bycity_clean.csv', index=False)
temps_bycountry.to_csv('../climate_final/temps_bycountry_clean.csv', index=False)
q1_df.to_csv('../climate_final/question1_df.csv', index=False)
q2_df.to_csv('../climate_final/question2_df.csv', index=False)
q3_df.to_csv('../climate_final/question3_df.csv', index=False)
q4_df.to_csv('..climate_final/question4_df.csv', index=False)